## Project Structure

In [1]:
import sys
sys.version

'3.9.1 (default, May 28 2021, 21:38:31) \n[GCC 9.3.0]'

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Zindi: PAYGo

In [3]:
train = pd.read_csv("../../data/Train.csv", sep=",")
metadata = pd.read_csv("../../data/metadata.csv", sep=",")
test = pd.read_csv("../../data/Test.csv", sep=",")

/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
metadata[metadata["ID"].isin(list(train["ID"].values))]

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52
6,ID_6YU5YRB,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,56.0,Mount Kenya Region,Meru,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 15:15:16,2016-12-11 11:48:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37338,ID_OKJ96OW,2020-02-20 18:26:52,2400,NaN,0.0,FINANCED,DAILY,50,3,Female,38.0,Western,Bungoma,Farmer,d_light,240,14400.0,2020-10-17 18:26:52,2020-02-20 18:26:58,2020-12-19 07:16:44
37339,ID_5J9OMUP,2020-02-20 18:30:59,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,23.0,Nyanza,Siaya,Labourer,d_light,240,14400.0,2020-10-17 18:30:59,2020-02-20 18:31:06,2021-05-18 16:07:49
37340,ID_J4I8Z2V,2020-02-20 18:42:26,3000,NaN,0.0,FINANCED,DAILY,40,3,Male,31.0,Coast Region,Kwale,Teacher,d_light,340,16600.0,2021-01-25 18:42:26,2020-02-20 18:42:33,2021-04-21 15:59:37
37341,ID_GHHAQ9D,2020-02-20 18:46:30,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,27.0,NaN,UNKNOWN,Labourer,d_light,240,14400.0,2020-10-17 18:46:30,2020-02-20 18:46:37,2021-04-24 19:06:01


In [9]:
# Merge
train_merged = metadata.merge(train[train.columns], how='right', on='ID')

In [10]:
print(train["ID"].nunique())         # Each contract is unique
print(metadata["ID"].nunique())      # Each contract is unique

28007
37343


In [11]:
metadata.shape

(37343, 20)

In [12]:
train_merged.iloc[:, :]

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,...,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_MR53LEX,2018-04-23 12:28:05,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,2018-04-23 12:28:05,2021-05-26 17:14:45,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0...",880.0,930.0,495.0,715.0,220.0,385.0
1,ID_3D7NQUH,2018-04-17 10:27:35,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,2018-04-17 10:27:35,2021-05-26 16:54:53,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 11...",660.0,935.0,935.0,825.0,770.0,935.0
2,ID_0IWQNPI,2020-02-18 14:23:44,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,...,2020-02-18 14:23:48,2021-01-25 18:46:20,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]",700.0,1350.0,1550.0,1400.0,1450.0,1200.0
3,ID_IY8SYB9,2017-09-14 11:07:40,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,...,2017-09-14 11:07:32,2018-12-20 16:50:39,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0,...",580.0,480.0,800.0,1260.0,1650.0,530.0
4,ID_9XHL7VZ,2017-09-06 06:50:20,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,...,2017-09-06 06:50:16,2021-05-12 14:50:01,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 33...",40.0,440.0,460.0,360.0,80.0,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28002,ID_CDZ24L9,2018-03-24 09:52:09,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,2018-03-24 09:52:13,2019-10-11 12:37:41,"['03-2018', '04-2018', '05-2018', '06-2018', '...","[5235.0, 1270.0, 940.0, 1370.0, 2020.0, 1750.0...",1640.0,1550.0,2045.0,850.0,1730.0,720.0
28003,ID_0XINELS,2020-02-17 13:34:57,2400,NaN,0.0,FINANCED,DAILY,50,3,Female,...,2020-02-17 13:35:01,2021-03-13 14:23:22,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2980.0, 1000.0, 450.0, 1050.0, 850.0, 1250.0,...",900.0,950.0,650.0,750.0,1200.0,620.0
28004,ID_PAU9JJU,2020-01-15 11:12:06,2000,NaN,0.0,FINANCED,DAILY,40,3,Male,...,2020-01-15 11:12:11,2021-03-28 16:01:16,"['01-2020', '02-2020', '03-2020', '04-2020', '...","[2580.0, 1020.0, 540.0, 360.0, 200.0, 660.0, 1...",680.0,460.0,799.0,1070.0,820.0,401.0
28005,ID_K866QHS,2018-02-25 16:51:21,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,...,2018-02-25 16:53:11,2019-03-20 16:38:09,"['02-2018', '03-2018', '04-2018', '05-2018', '...","[2000.0, 1160.0, 1240.0, 1120.0, 1000.0, 1200....",1300.0,1055.0,1200.0,1200.0,1080.0,800.0


In [13]:
train_merged["SupplierName"].unique()  # Can discard as a feature

array(['d_light'], dtype=object)

In [15]:
# train_merged['days_elapsed_after_term_completion'] = 

In [15]:
train_merged["ID"].nunique()

28007

In [16]:
# Creating a baseline model to get the base RMSE score
# Creating an XGBoost model and predicting M1 only at the moment for completing the pipeline

In [17]:
# Identifying the categorical columns
train_merged.select_dtypes(exclude=['number']).columns

Index(['ID', 'RegistrationDate', 'UpsellDate', 'PaymentMethod',
       'rateTypeEntity', 'MainApplicantGender', 'Region', 'Town', 'Occupation',
       'SupplierName', 'ExpectedTermDate', 'FirstPaymentDate',
       'LastPaymentDate', 'TransactionDates', 'PaymentsHistory'],
      dtype='object')

In [18]:
# Identifying the numerical columns
train_merged.select_dtypes(exclude=['object']).columns

Index(['Deposit', 'AccessoryRate', 'RatePerUnit', 'DaysOnDeposit', 'Age',
       'Term', 'TotalContractValue', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6'],
      dtype='object')

### Visualising some interesting distributions

In [23]:
train_merged[train_merged['MainApplicantGender'] == 'Male'][['TotalContractValue']].describe()

,TotalContractValue
count,17483.000000
mean,23256.360522
std,10329.956762
min,10800.000000
25%,14400.000000
50%,16560.000000
75%,33080.000000
max,64484.000000


In [20]:
train_merged[train_merged['MainApplicantGender'] == 'Female'][['TotalContractValue']].describe()

,TotalContractValue
count,10524.000000
mean,22473.612885
std,9871.017043
min,10800.000000
25%,16560.000000
50%,16560.000000
75%,33080.000000
max,64484.000000


In [21]:
train_merged['MainApplicantGender'].value_counts()

Male      17483
Female    10524
Name: MainApplicantGender, dtype: int64

In [25]:
train_merged['rateTypeEntity'].value_counts()  # Daily is more

DAILY      27883
MONTHLY       67
WEEKLY        57
Name: rateTypeEntity, dtype: int64

In [27]:
train_merged['DaysOnDeposit'].value_counts()

3     17736
7     10114
60       67
10       57
0        31
14        2
Name: DaysOnDeposit, dtype: int64

In [60]:
# Seems majority Contracts are of the value 16,560 and coast region has some costly Contracts
train_merged[['Region', 'TotalContractValue']].groupby(['Region']).agg(
    {'TotalContractValue': [np.sum, np.mean, np.median, 'count']}).sort_values(
    by=[('TotalContractValue', 'sum')], ascending=False)

TotalContractValue                             
                                  sum          mean   median count
Region                                                            
Nyanza                    123985830.0  23625.348704  16560.0  5248
Coast Region               93598065.0  26880.547099  33080.0  3482
South Rift                 88292085.0  24369.882694  16560.0  3623
North Rift                 87107800.0  21771.507123  16560.0  4001
Nairobi Region             73774070.0  19444.931471  16560.0  3794
Western                    71291055.0  21092.028107  16560.0  3380
Mount Kenya Region         70415475.0  23216.444115  16560.0  3033

In [24]:
# Male and female contract value distribution is almost similar
px.box(train_merged, x= 'MainApplicantGender', y='TotalContractValue', points='all')

In [34]:
px.box(train_merged, x= 'Region', y='TotalContractValue')

### Regressor model

In [19]:
def predict_model(train_array, target):
    model = xgb.XGBRegressor(use_label_encoder=False,  # Will be deprecated
                             verbosity=0,
                             objective="reg:squarederror",
                             reg_lambda=0,
                             reg_alpha=50
                            )
    for i in range(1, 11):
        X_train, X_test, y_train, y_test = train_test_split(train_array, target, test_size=0.25)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        from sklearn.metrics import mean_absolute_percentage_error
        from sklearn.metrics import mean_squared_error
        feature_importance = model.feature_importances_
        # Mape
        mape = mean_absolute_percentage_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)

    return y_pred, mape, mse, feature_importance